In [1]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.cluster import KMeans

In [2]:
#Data Preparation
#user_feature table
user_feat = pd.DataFrame(np.random.randint(low=0, high=10, size=(100, 3)),columns=['Age','Sex','Location'])


user_feat['Age'] = pd.DataFrame(np.random.randint(5,100, size = (100,1)))
user_feat['Location'] = np.random.choice(['J&K', 'Haryana', 'Punjab', 'U.P.', 'Maharashtra', 'South India'],100)
user_feat['Sex']= np.random.choice(["M","F"],100)

num = np.arange(500,5000)
num = num[0:len(user_feat)]
user_feat['user_guid'] = num
print type(user_feat)

#spice_feature table
spice_feat = pd.DataFrame(np.random.randint(low=0, high=2, size=(100, 6)),columns=['spice1','spice2','spice3','spice4','count','Location'])
num = np.arange(60000,70000)
num = num[0:len(spice_feat)]

spice_feat['food_id'] = num
spice_feat['count'] = np.random.randint(0,5, size=len(spice_feat))
spice_feat['Location'] = np.random.choice(['J&K', 'Haryana', 'Punjab', 'U.P.', 'Maharashtra', 'South India'],100)

l_dict = {'J&K':2, 'Haryana':5, 'Punjab':6, 'U.P.':8, 'Maharashtra':10, 'South India':15}

spice_feat  = spice_feat.replace({'Location':l_dict})

#food user rating table
food_user = pd.DataFrame(np.random.randint(low=0, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
food_user.index  =  spice_feat['food_id']


<class 'pandas.core.frame.DataFrame'>


In [3]:
#Data Normalisation

def nor_age(age):
    
    if(age<=10):
        return 1
    elif(age>10 and age <=18):
        return 2
    elif(age>18 and age <=25):
        return 3
    elif(age>25 and age <=35):
        return 4
    elif(age>35 and age <=40):
        return 5
    elif(age>40 and age <=50):
        return 6
    elif(age>50 and age<=60):
        return 7
    else:
        return 8
    
    

s_dict = {"M":1,"F":5 }
l_dict = {'J&K':2, 'Haryana':5, 'Punjab':6, 'U.P.':8, 'Maharashtra':10, 'South India':15}
user_feat['Age'] = user_feat['Age'].apply(nor_age)
user_feat  = user_feat.replace({"Sex":s_dict})
user_feat  = user_feat.replace({"Location":l_dict})

In [4]:
print "enter the user id"

input_user_id = input()



enter the user id
555


In [5]:
kmeans = KMeans(n_clusters = 10,random_state=0).fit(user_feat)
user_feat['label'] = kmeans.labels_


index = np.where(user_feat['user_guid']==input_user_id)[0]
#print index
user_label = user_feat.iloc[index]['label'].tolist()[0]
user_feat = user_feat.sort(['label','Location'],ascending=[1,0])
user = user_feat.drop_duplicates(subset=['Location','label'])
user = user.groupby('label')['Location'].nlargest(3)
loc = user[user_label].tolist()
print loc

[10, 8, 6]


C:\Users\ravindra\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [6]:
print user





label    
0      42     8
       43     6
       40     5
1      78    15
       83    10
       79     8
2      3     15
       6     10
       0      8
3      66    15
       70    10
       71     8
4      28    15
       29    10
       27     8
5      48    15
       54    10
6      91    15
       95    10
       89     6
7      58    10
       55     8
       61     6
8      11    15
       16    10
       15     8
9      18     6
       17     5
       19     2
Name: Location, dtype: int64


In [7]:

newspice_feat = spice_feat.set_index('food_id')
spice = newspice_feat[newspice_feat['Location'].isin(loc)]
s_spice =  spice.sort('count',ascending=[0])
s_index = food_user.index[food_user[input_user_id]==0]
new_spices = s_spice[s_spice.index.isin(s_index)]
print new_spices

         spice1  spice2  spice3  spice4  count  Location
food_id                                                 
60025         1       1       1       0      4        10
60065         0       1       1       0      4         6
60078         0       1       0       1      4         8
60029         0       0       0       1      4         6
60033         1       1       1       0      4        10
60017         1       1       0       0      4        10
60067         1       1       0       0      3        10
60072         0       0       0       0      3         6
60094         1       0       0       1      3         8
60019         0       1       0       0      3        10
60086         0       1       1       1      3         6
60024         1       1       0       0      3         8
60087         1       0       0       0      2        10
60084         0       1       1       1      2         6
60071         0       1       1       0      2         6
60034         0       1       0

C:\Users\ravindra\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [8]:
recommend = []
for i in range(min(3,len(new_spices))):
    recommend.append(new_spices.index[i])
    
if(len(recommend)==0):
    other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]
    
    other_state_spice = other_state_spice.sort('count',ascending= [0])
    new_other_state = other_state_spice[other_state_spice.index.isin(s_index)]

    for i in range(min(3,len(new_other_state))):
        recommend.append(new_other_state.index[i])

    
print recommend    

[60025, 60065, 60078]


In [9]:
spice = np.array(spice_feat.iloc[:,0:4])
user_df = pd.DataFrame(np.random.randint(low=-1, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)

user = np.array(user_df)
global spice
global user

In [10]:
# we have user_table which will give rating to a particular dish
# content based recommeder system
# we have 2 tables with us

#one is food - spices table and another is food-user rating table
#we will use tf-idf approach
#matrix of spice will contain only of 1 and 0's so need to normalize it

#normalization along the row
def spice_norm():
    
    num_attr = np.sum(spice,axis=1)
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0): 
            doc_vec.append(np.sqrt(spice[i,:]*1.0/num_attr[i]))
        else:
            doc_vec.append(spice[i,:])
    return np.array(doc_vec)      
    
    
    
def attr_norm():
    
    num_attr = np.sum(spice,axis=0)
    
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0):
            doc_vec.append(spice[:,i]*1.0 / num_attr[i])
        else:
            doc_vec.append(spice[:,i])
    return np.array(doc_vec)

def user_v(spice_mat):
    user_vec = []
    
    
    #print user[:,0]
    #print spice_mat[:,0]
    
    for i in range(user.shape[1]):
        user_vec.append([np.dot(user[:,i],spice_mat[:,j]) for j in range(spice_mat.shape[1])])
    
    print (user_vec)
    return np.array(user_vec)
    
def weight_predict(spice_mat):
        
    user_vec = user_v(spice_mat)
    
    preds = []
    #print spice_mat
    for i in range(spice_mat.shape[0]):
        preds.append([np.dot(spice_mat[i,:],user_vec[j,:]) for j in range(user_vec.shape[0])])

    return np.array(preds)
    
    
def idf_weight_pred():
    spice_mat = spice_norm()
    attr_wght = attr_norm()
    #print attr_wght
    weighted_spice = []
    
    for i in range(spice_mat.shape[0]):
        weighted_spice.append([np.dot(spice_mat[i,:],attr_wght[:,j]) for j in range(attr_wght.shape[0])])
        
    preds  = weight_predict(spice_mat = np.array(weighted_spice))     
    
    return np.array(preds)
    


preds = idf_weight_pred()

#print preds.shape

[[0.43172635268641391, 0.43172635268641391, 0.38505865761266178, 0.40435856104662338], [-0.087945912231369439, -0.087945912231369439, -0.10816549416176191, -0.13769973703608432], [0.1633980514297976, 0.1633980514297976, 0.22714630271459696, 0.17169507579477014], [0.0085097431833015824, 0.0085097431833015824, -0.017558883892139936, -0.031596334292809558], [-0.13945337205686331, -0.13945337205686331, -0.15340493952493134, -0.2207999033210401], [0.1679901377482278, 0.1679901377482278, 0.066970544864015547, 0.1868119248668727], [-0.17055893226291677, -0.17055893226291677, -0.11342960023189211, -0.15556957861962634], [0.0061144242932667792, 0.0061144242932667792, 0.11075516291655282, 0.0025444889109941249], [-0.15078454670618607, -0.15078454670618607, -0.02311446471239989, -0.077843046066401578], [-0.29242064310146659, -0.29242064310146659, -0.1652435980254402, -0.19799863624667019], [-0.160868387036222, -0.160868387036222, -0.19353859385248404, -0.052906925952047705], [-0.17154398216227457

In [11]:
outputdf = pd.DataFrame(data = preds,index = newspice_feat.index ,columns = user_df.columns)

s_index = food_user.index[food_user[input_user_id]==0]
output = outputdf[outputdf.index.isin(s_index)]
print user_df.columns
newoutput = output[input_user_id].sort_values(ascending = False)
print newoutput
food_to_recommend = list(newoutput[0:3].index)

print food_to_recommend

Int64Index([542, 546, 543, 545, 540, 541, 544, 547, 551, 578, 587, 583, 579,
            580, 582, 584, 585, 586, 581, 503, 505, 507, 506, 500, 504, 509,
            512, 501, 502, 510, 513, 508, 566, 570, 571, 573, 567, 569, 577,
            568, 574, 572, 575, 576, 528, 530, 539, 529, 538, 527, 533, 535,
            532, 531, 537, 534, 536, 548, 549, 550, 552, 557, 554, 591, 594,
            595, 589, 593, 596, 598, 592, 597, 588, 590, 599, 558, 555, 561,
            565, 559, 560, 564, 553, 556, 562, 563, 511, 514, 520, 522, 516,
            515, 518, 524, 526, 517, 521, 525, 519, 523],
           dtype='int64', name=u'user_guid')
food_id
60030    0.000000
60085    0.000000
60036    0.000000
60041    0.000000
60072    0.000000
60011   -0.002044
60019   -0.002044
60007   -0.003068
60040   -0.003068
60087   -0.003068
60024   -0.003615
60017   -0.003615
60067   -0.003615
60038   -0.005747
60065   -0.005747
60034   -0.005747
60006   -0.005747
60071   -0.005747
60073   -0.005747
60078   